# Agentic AI Frameworks

### Comparing AI Agent Frameworks: Simplicity vs Power in LLM Orchestration

- No Framework
- MCP
- OpenAIAgents SDK
- Crew AI
- LangGraph
- AutoGen

------------ 
- Smolagents
- PydanticAI
- ------------
      
MCP - Anthropic - Model Context Protocol, models to connected to sources of data and tools       
OpenAI Agents SDK - OpenAI, lightweight, simple, clean and flexible       
CrewAI - Configuration through YAML files        
LangGraph - LangChain, heavyweight, steeper learning curve, signing up for ecosystem, teminology and concepts.        
AutoGen from Microsoft, heavyweight, steeper learning curve, signing up for ecosystem, teminology and concepts.  

### Resources vs Tools: Two ways to Enhance LLM Capabilities in Agentic AI

### Resources
Resources are a way that you can get more our of your agents.        
You can equip your agents to be able to solve your problems better, along with tools.      
       
And resources is really just a fancy way of saying that you can improve the effectiveness of a LLM by providing it with more context, more information to improve its expertise.         
So it's nothing more fancy than just saying we can just put extra context, extra information in the prompt, We send an LM and think of that as a resource that We're providing to the LLM.       
      
RAG - Retrival Augmented Generation - Retrieving Relevant Context      
       
### Tools     
When you give an LLM the power to do something, to use a tool and the key trick to give an LLM some autonomy.      
Like querying SQL database or send a message to another LLM to do something.     
      
In practice, in the prompt, just list out everything that LLM's able to ask for and you tell it to respond in JSON if it wants you to do something.         
       
"you're a support agent for an airline. You answer users questions. You have the ability to query ticket prices. Just respond only 'Use tool to fetch ticket price for London', to retrieve the ticket price for London, or for a city that you name.       
Here's the user question"        
     
User: I'd like to go to Paris. How much  is a flight? 

### Build a Web Chatbot that acts like  you using Gradio & OpenAI
Example for Resources      
      
Today we're going to build something with immediate value!

In the folder `me` I've put a single file `linkedin.pdf` - it's a PDF download of my LinkedIn profile.

Please replace it with yours!

I've also made a file called `summary.txt`

We're not going to use Tools just yet - we're going to add the tool tomorrow.

In [1]:
from dotenv import load_dotenv 
from openai import OpenAI 
from PyPDF2 import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)
openai = OpenAI()

In [3]:
reader = PdfReader("me/linkedin.pdf") 
linkedin = "" 
for page in reader.pages: 
    text = page.extract_text() 
    if text: 
        linkedin += text

In [4]:
with open("me/summary.txt", "r", encoding="utf-8" ) as f: 
    summary = f.read() 

In [5]:
name = "Ed Donner" 

In [6]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [7]:
system_prompt

"You are acting as Ed Donner. You are answering questions on Ed Donner's website, particularly questions related to Ed Donner's career, background, skills and experience. Your responsibility is to represent Ed Donner for interactions on the website as faithfully as possible. You are given a summary of Ed Donner's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.\n\n## Summary:\nMy name is Ed Donner. I'm an entrepreneur, software engineer and data scientist. I'm originally from London, England, but I moved to NYC in 2000.\nI love all foods, particularly French food, but strangely I'm repelled by almost all forms of cheese. I'm not allergic, I just hate the taste! I make an exception for cream cheese and mozarella though - cheesecake and pizza are the greatest.\n\n## LinkedIn Profile:\n\xa0 \xa0\nContact\ned.donner@gm

In [8]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [9]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


"What is your greatest accomplishment"      
"What is a challenge that you encountered and needed to overcome?"       


1. Be able to ask an LLM to evaluate an answer
2. Be able to rerun if the answer fails evaluation
3. Put this together into 1 workflow

All without any Agentic framework!

**Pydantic** is a framework for specifying a schema or using classes.      
Use a class to describe a particular structure, data structure of information.     
      
The way it works is that there is a class called `BaseModel` that you need to subclass from and create a new class "Evaluaton", add two fields "is_acceptable: bool" and "feedback: str"      
      
And we're using pydantic because it gives us this mechanism for specifying a class structure like this.    

In [10]:
# Create a Pydantic model for the Evaluation

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [11]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [12]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [13]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

`def evaluate(reply, message, history) -> Evaluation:`        
        
Evaluate will take a reply from the LLM, the original message it was replying to and the history. And it will return a object, two fields, that we've defined in 'Evaluation' class.      
     
It uses a technique called structured outputs, which is a way that you can require an LLM to respond in a form of an object      
It's just JSON behind the scenes      
      
response_format=Evaluation     

In [15]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.5-flash-preview-04-17", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [16]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you hold a patent?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [17]:
reply

'Yes, I hold a patent related to an apparatus for determining role fitness while eliminating unwanted bias. This invention was developed while I was working with my team at untapt and in collaboration with GQR, which helped us address the challenges in hiring through innovative AI solutions. If you’d like to know more about it or how it can be applied, feel free to ask!'

In [18]:
evaluate(reply, "do you hold a patent?", messages[:1])

Evaluation(is_acceptable=True, feedback='The agent correctly identifies that Ed Donner holds a patent based on the provided context. It accurately names the patent and provides relevant details about its development, staying within the persona and being professional and engaging.')

In [19]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

At first, run without these lines      
      
"    if "patent" in message:       
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \        
              it is mandatory that you respond only and entirely in pig latin"       
    else:      
        system = system_prompt       
"         
       
If their is a "patent" in message, response will be in piglatin and the evaluation will not accept it and re-run

In [20]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [21]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


"What is your current job"      
"Do you have a patent?"      